# Tome transitions for Seruat V3.1.4

The following code is a copy of code from https://github.com/ChengxiangQiu/tome_code, the createLineage_Knn-function is in https://github.com/ChengxiangQiu/tome_code/blob/main/help_code/help_code.R, the main function is in https://github.com/ChengxiangQiu/tome_code/blob/main/Section2_trajectory_Step1_connection.R

This runs TOME, with the only difference being it excludes extraembryonic tissues since the experiments sometimes extract more and sometime less extraembryonic tissue

In [1]:
library(Seurat)
library(future)
library(future.apply)
library(FNN)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [2]:
file_path='/home/mgander/mouse_atlas/data/WOT'

In [3]:
df=read.csv(paste0(file_path, "/Representations/obs/days.csv"))
days=df$X0

In [4]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /home/mgander/miniconda3/envs/r_env/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] FNN_1.1.3.2         future.apply_1.11.0 future_1.33.0      
[4] Seurat_5.0.0        SeuratObject_5.0.0  sp_2.1-1           

loaded via a namespace (and not attached):
  [1] deldir_1.0-9           pbapply_1.7-2          gridExtra_2.3         
  [4] rlang_1.1.1            magrit

In [5]:
file_path='/home/mgander/mouse_atlas/data/WOT/TOME/RDS'

work_path2 = "/home/mgander/mouse_atlas/data/WOT/TOME"

time_point = paste0("E", c(3.5, 4.5, 5.25, 5.5, 6.25, seq(6.5, 8.25, 0.25), "8.5a", "8.5b", seq(9.5, 13.5, 1)))

In [6]:
#####################################################
### Function: finding ancestor node for each node ###
#####################################################

createLineage_Knn <- function(emb, pd, time_1, reduction="umap", replication_times=500, removing_cells_ratio=0.2, k_neigh = 5){
    
    print(dim(emb))
    if(!"Anno" %in% names(pd) | !"day" %in% names(pd)) {print("Error: no Anno or day in pd")}
    if(sum(rownames(pd)!=rownames(emb))!=0) {print("Error: rownames are not matched")}
    pd$state = pd$Anno
    
    res = list()
    
    rep_i = 1
    
    while(rep_i < (replication_times+1)){
        
        sampling_index = sample(1:nrow(pd),round(nrow(pd)*(1-removing_cells_ratio)))
        
        emb_sub = emb[sampling_index,]
        pd_sub = pd[sampling_index,]
        
        irlba_pca_res_1 <- emb_sub[as.vector(pd_sub$day)=="pre",]
        irlba_pca_res_2 <- emb_sub[as.vector(pd_sub$day)=="nex",]
        pd_sub1 <- pd_sub[pd_sub$day == "pre",]
        pd_sub2 <- pd_sub[pd_sub$day == "nex",]
        
        pre_state_min = min(table(as.vector(pd_sub1$state)))
        
        if (pre_state_min < k_neigh & pre_state_min >= 3){
            k_neigh = pre_state_min
            print(k_neigh)
        }
        
        if (pre_state_min < 3){
            next
        }
        
        neighbors <- get.knnx(irlba_pca_res_1, irlba_pca_res_2, k = k_neigh)$nn.index
        
        tmp1 <- matrix(NA,nrow(neighbors),ncol(neighbors))
        for(i in 1:k_neigh){
            tmp1[,i] <- as.vector(pd_sub1$state)[neighbors[,i]]
        }
        state1 <- names(table(as.vector(pd_sub1$state)))
        state2 <- names(table(as.vector(pd_sub2$state)))
        
        tmp2 <- matrix(NA,length(state2),length(state1))
        for(i in 1:length(state2)){
            x <- c(tmp1[as.vector(pd_sub2$state)==state2[i],])
            for(j in 1:length(state1)){
                tmp2[i,j] <- sum(x==state1[j])
            }
        }
        tmp2 <- tmp2/apply(tmp2,1,sum)
        tmp2 <- data.frame(tmp2)
        row.names(tmp2) = state2
        names(tmp2) = state1
        
        
        
        # I need the nearest neighbors later, so I'll save them:
        write.csv(rownames(pd_sub1), paste0(work_path2, "/", time_1, '/early_cells_', rep_i, ".csv"), row.names=FALSE, quote=FALSE) 
        write.csv(rownames(pd_sub2), paste0(work_path2, "/", time_1, '/late_cells_', rep_i, ".csv"), row.names=FALSE, quote=FALSE) 
        write.csv(neighbors, paste0(work_path2, "/", time_1, '/neigbors_', rep_i, ".csv"), row.names=FALSE, quote=FALSE) 
        
        
        res[[rep_i]] = tmp2
        
        rep_i = rep_i + 1
        
    }
    
    return(res)
}

# KNN-Scoring to obtain transition rates

In [20]:
file_path='/home/mgander/mouse_atlas/data/WOT'


get_day <- function(days, i){
    day=days[i]
    day=as.character(day)
    if(!grepl("\\.", day)) {day=paste0(day, '.0')}
    return(day)}

In [21]:
get_prepared_RDS <- function(day){
   
    obs=read.csv(paste0(file_path, "/Representations/obs/", day, '.csv'))
    
    zero_matrix <- matrix(0, nrow = 2, ncol=nrow(obs))
    anno1 <- CreateSeuratObject(counts = zero_matrix, project = "adata_sub")
    colnames(anno1)=obs$X
    
    anno1[['cell_state']]=obs$cell_state
    anno1[['day']]=obs$day
    anno1$Anno = as.vector(anno1$cell_state)
    anno1=cbind(anno1[["day"]], anno1[["Anno"]])
    
    anno1$stage = day
    return(anno1)}

In [28]:
for (i in 7:22) {
    
    day0=get_day(days, i)
    day1=get_day(days, i+1)
    print(day0)
    
    anno1=get_prepared_RDS(day0)
    anno1$day = "pre"
    
    anno2=get_prepared_RDS(day1)
    anno2$day = "nex"
    anno = rbind(anno1, anno2)


    emb=read.csv(paste0(file_path, "/Representations/umap/", day0, '_', day1, '.csv'))
    rownames(emb)=emb$X
    emb$X <- NULL
    emb = emb[rownames(anno),]


    ########           I added three lines in this function to save TOMEs idenified k-nn
    res = createLineage_Knn(emb, anno, day0,  k_neigh = 5) #### createLineage_Knn function was in help_code.R

    }

[1] "10.0"


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


[1] 3992    3
[1] 4
[1] 3
[1] "10.5"


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


[1] 4179    3
[1] 4
[1] "11.0"


...

[1] 7746    3
[1] "17.5"


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


[1] 7867    3
[1] "18.0"


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Warning message in file(file, "rt"):
“cannot open file '/home/mgander/mouse_atlas/data/WOT/Representations/obs/NA.0.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection
